some language to register the post ???

In [13]:
# QAT
# https://pytorch.org/docs/stable/quantization.html#quantization-aware-training-for-static-quantization

import torch

# define a floating point model where some layers could benefit from QAT
class M(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # QuantStub converts tensors from floating point to quantized
        self.quant = torch.quantization.QuantStub()
        self.conv = torch.nn.Conv2d(1, 1, 1)
        self.bn = torch.nn.BatchNorm2d(1)
        self.relu = torch.nn.ReLU()
        # DeQuantStub converts tensors from quantized to floating point
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.dequant(x)
        return x


In [21]:
# create a model instance
model_fp32 = M()
model_fp32

M(
  (quant): QuantStub()
  (conv): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
  (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (dequant): DeQuantStub()
)

In [22]:
from copy import deepcopy

model = deepcopy(model_fp32)

model.qconfig = torch.quantization.get_default_qconfig('fbgemm')  # QAT config

In [19]:
# model must be set to eval for fusion to work
model.eval()

# fusions: [Conv, Relu], [Conv, BatchNorm], [Conv, BatchNorm, Relu], [Linear, Relu]
torch.quantization.fuse_modules(model, [['conv', 'bn', 'relu']], inplace=True)
model  # note 'Identity()'' where 'bn' and 'relu' modules were

M(
  (quant): QuantStub()
  (conv): ConvReLU2d(
    (0): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
  )
  (bn): Identity()
  (relu): Identity()
  (dequant): DeQuantStub()
)

In [23]:
# back to train for QAT
model.train()

torch.quantization.prepare_qat(model, inplace=True)

# train ...

model.eval()
torch.quantization.convert(model, inplace=True)
model

M(
  (quant): Quantize(scale=tensor([1.]), zero_point=tensor([0]), dtype=torch.quint8)
  (conv): QuantizedConv2d(1, 1, kernel_size=(1, 1), stride=(1, 1), scale=1.0, zero_point=0)
  (bn): QuantizedBatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (dequant): DeQuantize()
)

In [63]:
# quantize resnet
# https://pytorch.org/blog/quantization-in-practice/#quantization-aware-training-qat

import torchvision 

res_model = torchvision.models.resnet18()
res_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [73]:
from collections import OrderedDict

model = deepcopy(res_model)

model = torch.nn.Sequential(
    OrderedDict([
        ('quant', torch.quantization.QuantStub()),
        ('resnet', model),
        ('dequant', torch.quantization.DeQuantStub())
    ])
)

model.qconfig = torch.quantization.get_default_qconfig('fbgemm')  # QAT config

model

Sequential(
  (quant): QuantStub()
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.

In [74]:
# blocks: conv1-bn1-relu, conv2-bn2
# downsample: 0-1

model.eval()

torch.quantization.fuse_modules(model.resnet, [['conv1', 'bn1', 'relu']], inplace=True)

for k1, m1 in model.resnet.named_children():
    if "layer" in k1:  # in a sequential layer with basic blocks
        for k2, m2 in m1.named_children():
            torch.quantization.fuse_modules(m2, [["conv1", "bn1", "relu"], ["conv2", "bn2"]], inplace=True)
            # fuse downsample layers for skip connection if present
            for k3, m3 in m2.named_children():
                if "downsample" in k3:
                    torch.quantization.fuse_modules(m3, [["0", "1"]], inplace=True)
                        
model

Sequential(
  (quant): QuantStub()
  (resnet): ResNet(
    (conv1): ConvReLU2d(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (1): ReLU(inplace=True)
    )
    (bn1): Identity()
    (relu): Identity()
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): ConvReLU2d(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
        )
        (bn1): Identity()
        (relu): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): Identity()
      )
      (1): BasicBlock(
        (conv1): ConvReLU2d(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
        )
        (bn1): Identity()
        (relu): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), pad

In [75]:
# back to train for QAT
model.train()

torch.quantization.prepare_qat(model, inplace=True)

# train ...

model.eval()
torch.quantization.convert(model, inplace=True)
model

C:\Users\chris\AppData\Local\Continuum\anaconda3\envs\blog_env\lib\site-packages\torch\ao\quantization\observer.py:216: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."
C:\Users\chris\AppData\Local\Continuum\anaconda3\envs\blog_env\lib\site-packages\torch\ao\quantization\observer.py:1206: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  Returning default scale and zero point "


Sequential(
  (quant): Quantize(scale=tensor([1.]), zero_point=tensor([0]), dtype=torch.quint8)
  (resnet): ResNet(
    (conv1): QuantizedConvReLU2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=1.0, zero_point=0, padding=(3, 3))
    (bn1): Identity()
    (relu): Identity()
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1))
        (bn1): Identity()
        (relu): Identity()
        (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1))
        (bn2): Identity()
      )
      (1): BasicBlock(
        (conv1): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1))
        (bn1): Identity()
        (relu): Identity()
        (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), s

In [79]:
# use quantized resnet as backbone to FastRCNN...
# https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html#modifying-the-model-to-add-a-different-backbone

# make last linear layer (and avg pool ???) identity since no classification head
model.resnet.fc = torch.nn.Identity()

# follow same setup as 
# torchvision.models.detection.fasterrcnn_resnet50_fpn
# https://pytorch.org/vision/main/models/generated/torchvision.models.detection.fasterrcnn_resnet50_fpn.html#fasterrcnn-resnet50-fpn
# https://pytorch.org/vision/main/_modules/torchvision/models/detection/faster_rcnn.html#fasterrcnn_resnet50_fpn

# which builds FasterRCNN
# https://github.com/pytorch/vision/blob/main/torchvision/models/detection/faster_rcnn.py#L43

# and takes as input BackboneWithFPN which wraps ResNet (quantized)
# https://github.com/pytorch/vision/blob/main/torchvision/models/detection/backbone_utils.py#L118

In [83]:
# weights = FasterRCNN_ResNet50_FPN_Weights.verify(weights)
# weights_backbone = ResNet50_Weights.verify(weights_backbone)

# if weights is not None:
#     weights_backbone = None
#     num_classes = _ovewrite_value_param("num_classes", num_classes, len(weights.meta["categories"]))
# elif num_classes is None:
#     num_classes = 91

# is_trained = weights is not None or weights_backbone is not None
# trainable_backbone_layers = _validate_trainable_layers(is_trained, trainable_backbone_layers, 5, 3)
# norm_layer = misc_nn_ops.FrozenBatchNorm2d if is_trained else nn.BatchNorm2d

# backbone = resnet50(weights=weights_backbone, progress=progress, norm_layer=norm_layer)
# backbone = _resnet_fpn_extractor(backbone, trainable_backbone_layers)
# model = FasterRCNN(backbone, num_classes=num_classes, **kwargs)

# if weights is not None:
#     model.load_state_dict(weights.get_state_dict(progress=progress, check_hash=True))
#     if weights == FasterRCNN_ResNet50_FPN_Weights.COCO_V1:
#         overwrite_eps(model, 0.0)

# model